In [1]:
import pdb
from math import log10, floor

import numpy as np
import scipy.optimize as op
import scipy.special as spec
from scipy import stats
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import seaborn as sns
import corner

import astropy
from astropy import stats as astrostats

import emcee
import radvel

%matplotlib inline

In [36]:
sys_props   = pd.read_csv('legacy_tables/system_props_719.csv')
stellar     = pd.read_csv('/Users/lee/Academics/Astronomy/Planets/legacy_analysis/legacy_tables/legacy_specmatch_medians.csv')
masses_only = pd.read_csv('legacy_tables/planet_list_719.csv')

### Replace NaNs with dummy values, for now.

In [37]:
stellar = stellar.fillna(100.5)

In [30]:
stellar.iso_radius

0       10.872633
1        0.995520
2        0.847684
3        1.134055
4        0.705415
5        3.521337
6        1.042461
7        0.883259
8        0.922064
9        0.877352
10       0.541821
11       0.888111
12       1.315520
13       0.738911
14       0.785043
15       1.182713
16       0.609717
17       3.509724
18       0.820426
19       0.740713
20       1.799408
21       1.475974
22       0.941694
23       1.148642
24       0.983059
25       1.695650
26       0.708423
27       1.326872
28       0.873234
29       1.189164
          ...    
715      0.757601
716    100.000000
717      0.552128
718      0.619479
719      0.611346
720      0.490483
721      0.629802
722      0.656603
723      0.564297
724      0.573959
725      0.750085
726      0.561458
727      0.621757
728      0.513376
729      0.598386
730      0.582023
731      0.551040
732      0.641038
733      0.740958
734      0.561664
735      0.659810
736      0.551882
737      0.569042
738      0.656584
739      0

# Make table for planet parameters.

In [3]:
masses_real = masses_only.query('status != "N" and status != "A"').reset_index(drop=True)
masses_good = masses_real.query('mass != "nan"').reset_index(drop=True)#[masses_only.hostname != 'hip63510']
masses_s = masses_good.query('status == "S" or status == "SS"').reset_index(drop=True)
masses_planet = masses_good.query('status != "S" and status != "SS"').reset_index(drop=True)
masses_old  = masses_planet.query('status == "K"').reset_index(drop=True)
masses_new  = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)

In [4]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_planet['hostname']
masses_table['Msini (MJup)'] = masses_planet['mass_med']
masses_table['a (AU)'] = masses_planet['axis_med']
masses_table['P (d)'] = masses_planet['per_med']
masses_table['e'] = masses_planet['e_med']

names = np.unique(np.array(masses_table.Name))

nplanets = np.array(masses_table.groupby('Name').count()['e'])
db_nplanets = pd.DataFrame({'Name':names, 'nplanets':nplanets})
masses_table = pd.merge(masses_table, db_nplanets, on='Name').reset_index()

In [5]:
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_planet.hostname))
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1

In [40]:
def round_to_1(x):
    if np.isnan(x):
        return 0.0001
    else:
        return round(x, -int(floor(log10(x))))

def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x)))) - 1) 
round_sig_vec = np.vectorize(round_sig)
def round_to_reference(x, y):
    return round(x, -int(floor(log10(abs(y)))) + 1)
round_to_ref_vec = np.vectorize(round_to_reference)

In [23]:
masses_planet['Name'] = masses_table['Name']

# Round errors and parameters with significant figures.
masses_planet['mass_err_plus_r']  = round_sig_vec(np.array(masses_planet['mass_plus'] - masses_planet['mass_med']))
masses_planet['mass_err_minus_r'] = round_sig_vec(np.array(masses_planet['mass_med'] - masses_planet['mass_minus']))
masses_planet['mass_err_av']      = round_sig_vec(0.5*(masses_planet['mass_err_plus_r'] + masses_planet['mass_err_minus_r']))
masses_planet['mass_med_r']       = round_to_ref_vec(np.array(masses_planet['mass_med']), masses_planet['mass_err_av'])

masses_planet['axis_err_plus_r']  = round_sig_vec(np.array(masses_planet['axis_plus'] - masses_planet['axis_med']))
masses_planet['axis_err_minus_r'] = round_sig_vec(np.array(masses_planet['axis_med'] - masses_planet['axis_minus']))
masses_planet['axis_err_av']      = round_sig_vec(0.5*(masses_planet['axis_err_plus_r'] + masses_planet['axis_err_minus_r']))
masses_planet['axis_med_r']       = round_to_ref_vec(np.array(masses_planet['axis_med']), masses_planet['axis_err_av'])

masses_planet['per_err_plus_r']   = round_sig_vec(np.array(masses_planet['per_plus'] - masses_planet['per_med']))
masses_planet['per_err_minus_r']  = round_sig_vec(np.array(masses_planet['per_med'] - masses_planet['per_minus']))
masses_planet['per_err_av']       = round_sig_vec(0.5*(masses_planet['per_err_plus_r'] + masses_planet['per_err_minus_r']))
masses_planet['per_med_r']        = round_to_ref_vec(np.array(masses_planet['per_med']), masses_planet['per_err_av'])

masses_planet['e_err_plus_r']     = round_sig_vec(np.array(masses_planet['e_plus'] - masses_planet['e_med']))
masses_planet['e_err_minus_r']    = round_sig_vec(np.array(masses_planet['e_med'] - masses_planet['e_minus']))
masses_planet['e_err_av']         = round_sig_vec(0.5*(masses_planet['e_err_plus_r'] + masses_planet['e_err_minus_r']))
masses_planet['e_med_r']          = round_to_ref_vec(np.array(masses_planet['e_med']), masses_planet['e_err_av'])
'''
masses_planet['tc_err_plus_r']     = round_sig_vec(np.array(masses_planet['tc_plus'] - masses_planet['tc_med']))
masses_planet['tc_err_minus_r']    = round_sig_vec(np.array(masses_planet['tc_med'] - masses_planet['tc_minus']))
masses_planet['tc_err_av']         = round_sig_vec(0.5*(masses_planet['tc_err_plus_r'] + masses_planet['tc_err_minus_r']))
masses_planet['tc_med_r']          = round_to_ref_vec(np.array(masses_planet['tc_med']), masses_planet['tc_err_av'])

masses_planet['w_err_plus_r']     = round_sig_vec(np.array(masses_planet['w_plus'] - masses_planet['w_med']))
masses_planet['w_err_minus_r']    = round_sig_vec(np.array(masses_planet['w_med'] - masses_planet['w_minus']))
masses_planet['w_err_av']         = round_sig_vec(0.5*(masses_planet['w_err_plus_r'] + masses_planet['w_err_minus_r']))
masses_planet['w_med_r']          = round_to_ref_vec(np.array(masses_planet['w_med']), masses_planet['w_err_av'])
'''
tex_writer = open('legacy_tables/planet_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & \msini\ [\mjup] & a [AU] & $P$ [d] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in masses_planet.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['mass_med_r'], 
                                                   row['mass_err_plus_r'], 
                                                   row['mass_err_minus_r']) + r'$'
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['axis_med_r'], 
                                                   row['axis_err_plus_r'], 
                                                   row['axis_err_minus_r']) + r'$'
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['per_med_r'], 
                                                   row['per_err_plus_r'], 
                                                   row['per_err_minus_r']) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['e_med_r'], 
                                                   row['e_err_plus_r'], 
                                                   row['e_err_minus_r']) + r'$'
    '''
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['tc_med_r'], 
                                                 row['tc_err_plus_r'], 
                                                 row['tc_err_minus_r']) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(row['w_med_r'], 
                                                row['w_err_plus_r'], 
                                                row['w_err_minus_r']) + r'$'
    '''
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

# Make table for stellar parameters. TO-DO: COMBINE SYNTHETIC/EMPIRICAL SPECTRAL PARAMETERS, USE APPROPRIATE SET FOR EACH STAR. 

## THIS COMBINATION SHOULD BE DONE IN THE SPECMATCH SELECTION NOTEBOOK.

In [47]:
stellar.teff_err_emp

0      110.0
1      110.0
2      110.0
3      110.0
4      110.0
5      110.0
6      110.0
7      110.0
8      110.0
9      110.0
10     110.0
11     110.0
12     110.0
13      70.0
14     110.0
15     110.0
16      70.0
17     110.0
18     110.0
19     110.0
20     110.0
21     110.0
22     110.0
23     110.0
24     110.0
25     110.0
26     110.0
27     110.0
28     110.0
29     110.0
       ...  
715     70.0
716     70.0
717     70.0
718     70.0
719     70.0
720     70.0
721     70.0
722     70.0
723     70.0
724     70.0
725    110.0
726     70.0
727     70.0
728     70.0
729     70.0
730     70.0
731     70.0
732    110.0
733    110.0
734    110.0
735    110.0
736     70.0
737     70.0
738    110.0
739     70.0
740    110.0
741    110.0
742     70.0
743     70.0
744    110.0
Name: teff_err_emp, Length: 745, dtype: float64

In [48]:
# Specmatch parameters first. TO-DO: COMBINE SYNTHETIC/EMPIRICIAL RESULTS. MAKE COLUMN WITH SYN/EMP
stellar['teff_err_r'] = round_sig_vec(stellar['teff_err_emp'])
stellar['teff_r'] = round_to_ref_vec(np.array(stellar['teff_emp']), stellar['teff_err_r'])
stellar['fe_err_r'] = round_sig_vec(stellar['fe_err'])
stellar['fe_r'] = round_to_ref_vec(np.array(stellar['fe']), stellar['fe_err_r'])

# USE ISOCLASSIFY OUTPUTS FOR R AND LOGG, SINCE SPECMATCH-SYN AND -EMP EACH ONLY PRODUCE ONE. 
# FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX
stellar['logg_err_r'] = round_sig_vec(stellar['logg_err'])
stellar['logg_r'] = round_to_ref_vec(np.array(stellar['logg']), stellar['logg_err_r'])
stellar['radius_err_r'] = round_sig_vec(0.5*(stellar['iso_radius_err1'] + stellar['iso_radius_err2']))
stellar['radius_r'] = round_to_ref_vec(np.array(stellar['iso_radius']), stellar['radius_err_r'])

# Now isoclassify.
stellar['mass_err_r'] = round_sig_vec(0.5*(stellar['iso_mass_err1'] + stellar['iso_mass_err2']))
stellar['mass_r'] = round_to_ref_vec(np.array(stellar['iso_mass']), stellar['mass_err_r'])


tex_writer = open('legacy_tables/stellar_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & $T_{\textrm{eff}}$ [K] & [Fe/H] & log$g$ [log(cm s$^{-2}$)] & $M$ [\msun\] & $R$ [\rsun\] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in stellar.iterrows():
    # Customize names.
    namein = str(row['name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    teff_str   = r'$' + '{0}\\pm {1}'.format(row['teff_r'], row['teff_err_r']) + r'$'    
    fe_str     = r'$' + '{0}\\pm {1}'.format(row['fe_r'], row['fe_err_r']) + r'$'    
    logg_str   = r'$' + '{0}\\pm {1}'.format(row['logg_r'], row['logg_err_r']) + r'$' 
    radius_str = r'$' + '{0}\\pm {1}'.format(row['radius_r'], row['radius_err_r']) + r'$'      
    mass_str   = r'$' + '{0}\\pm {1}'.format(row['mass_r'], row['mass_err_r']) + r'$'

    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, teff_str, fe_str, logg_str, radius_str, mass_str))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()